# Intégration de Semantic Kernel avec le serveur MCP OpenBnB

Ce notebook montre comment utiliser Semantic Kernel avec le serveur MCP OpenBnB réel pour rechercher des hébergements Airbnb en utilisant le plugin MCPStdioPlugin. Pour l'accès au LLM, il utilise Azure AI Foundry. Pour configurer vos variables d'environnement, vous pouvez suivre la [leçon de configuration](/00-course-setup/README.md).


## Importer les Packages Nécessaires


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Création de la connexion au plugin MCP

Nous allons nous connecter au [serveur MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) en utilisant MCPStdioPlugin. Ce serveur offre des fonctionnalités de recherche Airbnb via le package @openbnb/mcp-server-airbnb.


## Création du Client

Dans cet exemple, nous utiliserons Azure AI Foundry pour accéder à notre LLM. Assurez-vous que vos variables d'environnement sont correctement configurées.


## Configuration de l'environnement

Configurez les paramètres d'Azure OpenAI. Assurez-vous d'avoir les variables d'environnement suivantes définies :
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Comprendre l'intégration OpenBnB MCP

Ce notebook se connecte au **véritable serveur OpenBnB MCP**, qui fournit des fonctionnalités de recherche Airbnb en temps réel.

### Comment cela fonctionne :

1. **MCPStdioPlugin** : Utilise la communication via entrée/sortie standard avec le serveur MCP
2. **Package NPM réel** : Télécharge et exécute `@openbnb/mcp-server-airbnb` via npx
3. **Données en direct** : Renvoie des données réelles sur les propriétés Airbnb à partir de leurs APIs
4. **Découverte de fonctions** : L'agent découvre automatiquement les fonctions disponibles sur le serveur MCP

### Fonctions disponibles :

Le serveur OpenBnB MCP propose généralement :
- **search_listings** - Rechercher des propriétés Airbnb par localisation et critères
- **get_listing_details** - Obtenir des informations détaillées sur des propriétés spécifiques
- **check_availability** - Vérifier la disponibilité pour des dates précises
- **get_reviews** - Récupérer les avis sur des propriétés
- **get_host_info** - Obtenir des informations sur les hôtes des propriétés

### Prérequis :

- **Node.js** installé sur votre système
- **Connexion Internet** pour télécharger le package du serveur MCP
- **NPX** disponible (inclus avec Node.js)

### Tester la connexion :

Vous pouvez tester manuellement le serveur MCP en exécutant :
```bash
npx -y @openbnb/mcp-server-airbnb
```

Cela téléchargera et démarrera le serveur OpenBnB MCP, auquel Semantic Kernel se connectera ensuite pour accéder aux données réelles d'Airbnb.


## Exécution de l'Agent avec le serveur OpenBnB MCP

Nous allons maintenant exécuter l'Agent IA qui se connecte au serveur OpenBnB MCP pour rechercher de véritables logements Airbnb à Stockholm pour 2 adultes et 1 enfant. N'hésitez pas à modifier la liste `user_inputs` pour ajuster les critères de recherche.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Résumé
Félicitations ! Vous avez réussi à créer un agent d'IA qui s'intègre à la recherche d'hébergements réels en utilisant le protocole Model Context Protocol (MCP) :

## Technologies utilisées :
- Semantic Kernel - Pour construire des agents intelligents avec Azure OpenAI
- Azure AI Foundry - Pour les capacités de modèles de langage (LLM) et la complétion de chat
- MCP (Model Context Protocol) - Pour une intégration standardisée des outils
- OpenBnB MCP Server - Pour une fonctionnalité de recherche Airbnb réelle
- Node.js/NPX - Pour exécuter le serveur MCP externe

## Ce que vous avez appris :
- Intégration MCP : Connecter des agents Semantic Kernel à des serveurs MCP externes
- Accès aux données en temps réel : Rechercher des propriétés Airbnb réelles via des API en direct
- Communication par protocole : Utiliser la communication stdio entre l'agent et le serveur MCP
- Découverte de fonctions : Découvrir automatiquement les fonctions disponibles des serveurs MCP
- Réponses en streaming : Capturer et enregistrer les appels de fonctions en temps réel
- Rendu HTML : Formater les réponses de l'agent avec des tableaux stylisés et des affichages interactifs

## Prochaines étapes :
- Intégrer des serveurs MCP supplémentaires (météo, vols, restaurants)
- Construire un système multi-agents combinant les protocoles MCP et A2A
- Créer des serveurs MCP personnalisés pour vos propres sources de données
- Mettre en œuvre une mémoire de conversation persistante entre les sessions
- Déployer l'agent sur Azure Functions avec orchestration de serveurs MCP
- Ajouter des fonctionnalités d'authentification utilisateur et de réservation



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
